<center>
<img src="https://image.aladin.co.kr/product/12609/37/cover500/k372532974_1.jpg" width="200" height="200"><br>
</center>

# 2장 서울시 범죄 현황 분석

이번에는 서울시의 범죄 현황 분석을 해보자. 구글에 '서울시 관서별 5대 범죄 발생 검거 현황' 이라는 키워드로 검색한 결과에 대한 데이터를 준비 하자. 자세한 데이터는 다음링크를 참고 하자.

 - Github: [PinkWink](https://github.com/PinkWink/DataScience)

### 2-1 pandas를 이용한 데이터 정리

먼저 numpy와 pandas를 import하자. 그리고 이제 이 데이터를 crime_anal_police로 저장 해놓자. 그리고 다음을 보면 서울시 경찰서별로 살인, 강도, 강간, 절도, 폭력이라는 5대 범죄에 대한 발생건수와 검거 건수를 갖고 있다.

In [13]:
import numpy as np
import pandas as pd
import json
import requests 


crime_anal_police = pd.read_csv("data/02. crime_in_Seoul.csv", thousands=',', 
                               encoding='euc-kr')

crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


여기서 문제가 하나 생긴다. 강남3구가 안전한지를 확인하려 할때 데이터가 관서별로 되어 있다. 서울시에는 한 구에 하나 혹은 두 군데의 경찰서가 위치해 있고, 구 이름과 다른 경찰서도 있다. 이 경찰서 목록을 소속 구별로 변경 하고 싶은데 이를 위해 이 정보를 알아야 한다. 이를 해결하기 위해 주소 데이터를 불러 오자.

### 2-1 Google Maps를 이용해서 주소와, 위도 경도 정보 얻기

위치에 대한 검색 결과중 주소와 위도, 경도 정보를 제공하는 서비스가 구글에 있다. Google Maps API라는건데 다음의 링크를 통해 Google Maps Geocoding API를 얻어주자.

In [19]:
# ! pip3 install googlemaps

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/29/c1/24814557f1d22c56d50280771a17307e6bf87b70727d975fd6b2ce6b014a/requests-2.25.1-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/19/c7/fa589626997dd07bd87d9269342ccb74b1720384a4d739a1872bd84fbe68/chardet-4.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a2/38/928ddce2273eaa564f6f50de919327bf3a00f091b5baba8dfa9460f3a8a8/idna-2.10-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/09/c6/d3e3abe5b4f4f16cf0dfc9240ab7ce10c2baa0e268989a4e3ec19e90c84e/urllib3-1.26.4-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/5e/a0/5f06e1e1d463903cf0c0eebeb751791119ed7a4b3737fdc9a77f

In [20]:
import googlemaps